In [13]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import json
from bson import json_util, ObjectId

from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool

load_dotenv()

client = MongoClient("mongodb://localhost:27017/")
db = client["learning_platform"]
courses_collection = db["courses"]
users_collection = db["users"]


@tool
def search_courses(query: str) -> str:
    """
    Use this tool ONLY for questions about course details like lessons, content, instructors, or descriptions.
    The input must be a valid Python command string using methods like find or aggregate on the 'courses_collection' variable.
    """
    print(f"  Executing on 'courses' collection: {query}")
    try:
        local_vars = {"courses_collection": courses_collection, "ObjectId": ObjectId}
        result = eval(query, {"__builtins__": {}}, local_vars)
        if isinstance(result, int): return str(result)
        result_list = list(result)
        if not result_list: return "No courses found matching the query."
        return json.dumps(result_list, default=json_util.default, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Query execution error: {str(e)}"


@tool
def search_users(query: str) -> str:
    """
    Use this tool ONLY for questions about users, their enrollments, or their progress in courses.
    The input must be a valid Python command string using methods like find or aggregate on the 'users_collection' variable.
    """
    print(f"  Executing on 'users' collection: {query}")
    try:
        local_vars = {"users_collection": users_collection, "ObjectId": ObjectId}
        result = eval(query, {"__builtins__": {}}, local_vars)
        if isinstance(result, int): return str(result)
        result_list = list(result)
        if not result_list: return "No users found matching the query."
        return json.dumps(result_list, default=json_util.default, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Query execution error: {str(e)}"


tools = [search_courses, search_users]

SYSTEM_MESSAGE = """
You are a highly specialized MongoDB assistant. Your only goal is to answer user questions by generating a complete, runnable Python command string to be executed by a tool.

**--- YOUR PRIMARY DIRECTIVE ---**
The input for your tools (`Action Input`) MUST be a Python string that starts with `users_collection.` or `courses_collection.`.
NEVER output just a JSON object. ALWAYS output the full command.
Correct format: `users_collection.find({{'full_name': 'Alice Johnson'}})`
Incorrect format: `{{'full_name': 'Alice Johnson'}}`

**--- AVAILABLE TOOLS and DATA ---**
You have two tools:
1. `search_courses`: Use for questions about courses, lessons, instructors, or descriptions. Queries MUST start with `courses_collection.`.
2. `search_users`: Use for questions about users, their enrollments, or progress. Queries MUST start with `users_collection.`.

**`courses` collection structure:**
{{
  "_id": "ObjectId",
  "title": "string",
  "instructor_name": "string",
  "description": "string",
  "lessons": [ {{ "title": "string", "content": "string" }} ]
}}

**`users` collection structure:**
{{
  "full_name": "string",
  "email": "string",
  "enrollments": [ {{ "course_id": "ObjectId", "progress": "number" }} ]
}}
Note: `enrollments.course_id` is a reference to the `_id` in the `courses` collection. To get course names for a user, you MUST use an `aggregate` query with `$lookup`.

**--- EXAMPLES ---**
Question: "Who is the instructor for 'Machine Learning Fundamentals'?"
Action: search_courses
Action Input: `courses_collection.find({{'title': 'Machine Learning Fundamentals'}}, {{'_id': 0, 'instructor_name': 1}})`

Question: "What is the content of the third lesson in the Python course?"
Action: search_courses
Action Input: `courses_collection.find({{'title': {{'$regex': 'python', '$options': 'i'}}}}, {{'_id': 0, 'lessons': {{'$slice': [2, 1]}}, 'lessons.content': 1}})`

Question: "What courses is Alice Johnson enrolled in and what is her progress?"
Action: search_users
Action Input: `users_collection.aggregate([{{'$match': {{'full_name': 'Alice Johnson'}}}}, {{'$unwind': '$enrollments'}}, {{'$lookup': {{'from': 'courses', 'localField': 'enrollments.course_id', 'foreignField': '_id', 'as': 'course_details'}}}}, {{'$unwind': '$course_details'}}, {{'$project': {{'_id': 0, 'course_title': '$course_details.title', 'progress': '$enrollments.progress'}}}}])`
---
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_MESSAGE),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)


def ask_agent(question):
    print(f"user question: {question}")
    response = agent_executor.invoke({"input": question})
    print("Agent response:")
    print(response["output"])

In [17]:
ask_agent("How many lessons does the 'Machine Learning Fundamentals' course have?")

user question: How many lessons does the 'Machine Learning Fundamentals' course have?


> Entering new AgentExecutor chain...

Invoking: `search_courses` with `{'query': "courses_collection.find({'title': 'Machine Learning Fundamentals'}, {'_id': 0, 'lessons': 1})"}`


  Executing on 'courses' collection: courses_collection.find({'title': 'Machine Learning Fundamentals'}, {'_id': 0, 'lessons': 1})
[
  {
    "lessons": [
      {
        "title": "Lesson 1: What is Machine Learning?",
        "content": "An overview of the field and its applications."
      },
      {
        "title": "Lesson 2: Linear Regression",
        "content": "Understanding and implementing a simple linear regression model."
      },
      {
        "title": "Lesson 3: Classification with Logistic Regression",
        "content": "Learn how to solve classification problems."
      }
    ]
  }
]The 'Machine Learning Fundamentals' course has 3 lessons.

> Finished chain.
Agent response:
The 'Machine Learning Fundame

In [16]:
ask_agent("How many courses are available in total?")
ask_agent("Who teaches the 'Introduction to Python' course?")
ask_agent("How many lessons does the 'Machine Learning Fundamentals' course have?")
ask_agent("Which course includes a lesson about 'if-else statements'?")
ask_agent("What are the titles of the courses Alice Johnson is enrolled in?")
ask_agent("Which students are enrolled in the course taught by Grace Hopper?")
ask_agent("What is Bob Williams's progress percentage in the 'Web Development with React' course?")
ask_agent("List the full names of students who have 100% progress in any course.")
ask_agent("Find courses that mention 'predictive models' in their description.")
ask_agent("List all the lesson titles for the 'Introduction to Python' course.")

user question: How many courses are available in total?


> Entering new AgentExecutor chain...

Invoking: `search_courses` with `{'query': 'courses_collection.count_documents({})'}`


  Executing on 'courses' collection: courses_collection.count_documents({})
3The total number of courses available is 3.

> Finished chain.
Agent response:
The total number of courses available is 3.
user question: Who teaches the 'Introduction to Python' course?


> Entering new AgentExecutor chain...

Invoking: `search_courses` with `{'query': "courses_collection.find({'title': 'Introduction to Python'}, {'_id': 0, 'instructor_name': 1})"}`


  Executing on 'courses' collection: courses_collection.find({'title': 'Introduction to Python'}, {'_id': 0, 'instructor_name': 1})
[
  {
    "instructor_name": "Dr. Ada Lovelace"
  }
]The 'Introduction to Python' course is taught by Dr. Ada Lovelace.

> Finished chain.
Agent response:
The 'Introduction to Python' course is taught by Dr. Ada Lovelace.
user questio